In [1]:
!pip install autoawq
!pip install nvidia-ml-py3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.3 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 42.1 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=8a01d8118f22a2ca6d49ba268fcfaaa50af48e3fcd2a7ffe1cd01a57a052d729
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [2]:
%%capture
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
from huggingface_hub import notebook_login as hfl

2024-05-26 16:14:03.869321: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 16:14:03.869439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 16:14:04.135954: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
hfl()

In [6]:
model_id = r"0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP"
quantModelPath = r"Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"
quantConfig = {
    "q_group_size": 128,
    "w_bit": 4
}

Accidentally used `AutoModelForCausalLM` as a consequence of muscle memory. Should be `AutoAWQForCausalLM` :)

In [7]:
model = AutoAWQForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

mergekit_config.yml:   0%|          | 0.00/405 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [8]:
%%time
model.quantize(tokenizer, quant_config=quantConfig)

Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100%|██████████| 32/32 [49:10<00:00, 92.20s/it]

CPU times: user 47min 51s, sys: 2min 23s, total: 50min 14s
Wall time: 49min 38s


In [9]:
%%time
model.save_quantized("./" + quantModelPath, safetensors=True)
tokenizer.save_pretrained("./" + quantModelPath)

CPU times: user 3.66 s, sys: 11 s, total: 14.7 s
Wall time: 13.8 s


('./Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit/tokenizer_config.json',
 './Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit/special_tokens_map.json',
 './Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit/tokenizer.json')

In [11]:
from kaggle_secrets import UserSecretsClient

userSecrets = UserSecretsClient()
HF_WRITE_TOKEN = userSecrets.get_secret("HF_WRITE_TOKEN")

In [12]:
from huggingface_hub import HfApi

username = "0xVolt"
modelName = "Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"
outputDir = r"/kaggle/working/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"

# Defined in the secrets tab in Kaggle Secrets
api = HfApi(token=HF_WRITE_TOKEN)

api.create_repo(
    repo_id=f"{username}/{modelName}",
    repo_type="model"
)

# Push the whole merged model's folder to the hub
api.upload_folder(
    repo_id=f"{username}/{modelName}",
    folder_path=outputDir,
)

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit/commit/a78c04bff1f2c953591ab0d453caf5de1b6ed7c6', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a78c04bff1f2c953591ab0d453caf5de1b6ed7c6', pr_url=None, pr_revision=None, pr_num=None)